# Imports and Setup

In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# CNN Architecture

In [6]:
class CNN_classifier(nn.Module):
    def __init__(self):
        super(CNN_classifier, self).__init__()

        self.conv1 = nn.Conv2d(3,5, kernel_size =5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(5,32, kernel_size=5)

        self.fc1 = nn.Linear(32*5*5, 50)
        self.fc2 = nn.Linear(50,10)


    def forward(self, x):
        # First conv + pool
        x = self.pool(F.relu(self.conv1(x)))
        # Second conv + pool
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten: [batch_size, 32*5*5]
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # final logits for 10 classes
        return x

In [7]:
model = CNN_classifier()
print(model)

CNN_classifier(
  (conv1): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


#  Loading the Data

In [8]:
# Transforms: convert to Tensor, then normalize.
# CIFAR-10 means 3-channel images in [0,1], we often normalize to e.g. mean=[0.5,0.5,0.5].
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10(root='data', train=True, 
                              download=True, transform=transform)
val_data   = datasets.CIFAR10(root='data', train=False, 
                              download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:03<00:00, 53735160.98it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


# Define Loss and Optimizer

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


# Training and Validation Loops

In [13]:
def train_one_epoch(model, data_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    
    for images, labels in data_loader:
        # 1) Zero the parameter gradients
        optimizer.zero_grad()
        # 2) Forward pass
        outputs = model(images)
        # 3) Compute loss
        loss = criterion(outputs, labels)
        # 4) Backprop
        loss.backward()
        # 5) Update the weights
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(data_loader)

def validate(model, data_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Argmax for predicted class
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return val_loss / len(data_loader), accuracy

# Training

In [14]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = validate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Accuracy: {val_accuracy*100:.2f}%")

Epoch 1/5, Train Loss: 1.4789, Val Loss: 1.3555, Val Accuracy: 51.13%
Epoch 2/5, Train Loss: 1.2746, Val Loss: 1.2456, Val Accuracy: 56.04%
Epoch 3/5, Train Loss: 1.1588, Val Loss: 1.1562, Val Accuracy: 59.28%
Epoch 4/5, Train Loss: 1.0661, Val Loss: 1.1385, Val Accuracy: 59.58%
Epoch 5/5, Train Loss: 1.0130, Val Loss: 1.1178, Val Accuracy: 61.38%
